In [1]:
import pandas as pd
import numpy as np
import random
from sympy import isprime, nextprime
import os
import pickle
os.chdir('Resources/')

In [2]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [4]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [5]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [6]:
df = pd.read_csv('4_CC_Structured_Data_Part.csv')

public_key = load_key("3_CC_Public_Key.pkl")

In [7]:
column_names = [
    "ST_Slope", "HeartDisease"
]

In [8]:
encrypted_data = pd.DataFrame()

public_data = {}

for k in range(1):
    p = public_key['p']
    g = public_key['g']
    h = public_key['h']
    
    public_data[column_names[k]] = {'p': p, 'g': g, 'h': h}
    
    print(f"Processing Column: {column_names[k]}")

    y = random.randint(1, p - 2)
    public_data[column_names[k]]['y'] = y
    
    s = power(h, y, p)

    messages = df.iloc[:, k].values 

    encrypted_column = []

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:  
            print(f"Processing Message {i}: {message}")
            
            temp_message = string_to_int(message)
            
            c1 = power(g, y, p)
            c2 = (temp_message * s) % p
            
            encrypted_column.append(f"({c1},{c2})") 
            print(f"Ciphertext: (c1, c2): ({c1}, {c2})")
        
        else:
            encrypted_column.append(None)

    encrypted_data[column_names[k]] = encrypted_column

df['HeartDisease'] = df['HeartDisease'].replace({'Yes': 1, 'No': 0})
combined_df = pd.concat([encrypted_data, df['HeartDisease']], axis=1)
combined_df.to_csv('5_3_CC_Encrypted_Data_Part.csv', index=False)

Processing Column: ST_Slope
Processing Message 1: Upsloping
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 5701636905427024824662866983306169439856314483175)
Processing Message 2: Flat
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 2743653045689028069856944757816058483274051618142)
Processing Message 3: Upsloping
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 5701636905427024824662866983306169439856314483175)
Processing Message 4: Flat
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 2743653045689028069856944757816058483274051618142)
Processing Message 5: Upsloping
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 5701636905427024824662866983306169439856314483175)
Processing Message 6: Upsloping
Ciphertext: (c1, c2): (33269922789787438881901292310454632547714019546100, 5701636905427024824662866983306169439856314483175)
Processing Message 7: Upsloping
Ci

C:\Users\USER\AppData\Local\Temp\ipykernel_17012\2561535717.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HeartDisease'] = df['HeartDisease'].replace({'Yes': 1, 'No': 0})


In [9]:
combined_df.to_csv('5_3_CC_Encrypted_Data_Part.csv', index=False)

with open('5_3_CC_Public_Key_Part.pkl', 'wb') as f:
    pickle.dump(public_data, f)